In [2]:
import os
import math
import cv2
import numpy as np
import warnings
import pandas as pd
warnings.filterwarnings(action='ignore')

In [3]:
from keras.applications import VGG16
from keras.models import Model

Using TensorFlow backend.


In [4]:
from keras.applications.vgg16 import VGG16, decode_predictions

vgg = VGG16(include_top=True, weights="imagenet")
vgg.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
bloc

In [5]:
fc1 = vgg.layers[-3]
fc2 = vgg.layers[-2]
predictions = vgg.layers[-1]

# Reconnect the layers
x = fc1.output
x = fc2(x)

# Create a new model
model = Model(input=vgg.input, output=x)

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [ ]:
TRAIN_DIR = 'image2/'
train_folder_list = os.listdir(TRAIN_DIR)

Total_input = []
Total_label = []

for index in train_folder_list:
    img = cv2.imread(TRAIN_DIR+index)
    ## print(index)
    ## print(img.shape)
    img = cv2.resize(img, dsize=(224,224), interpolation=cv2.INTER_AREA)
    img = img/255
    
    yhat = model.predict(img.reshape(-1,224,224,3))
    
    Total_input.append(yhat)
    Total_label.append(index)
    
print("complete")

In [121]:
for i in range(0,len(Total_label)):
    Total_label[i] = Total_label[i][0:len(Total_label[i])-4]

In [98]:
sample = np.array(Total_input)

In [99]:
sample = sample.reshape(sample.shape[0], sample.shape[2])

In [100]:
sample_sparse = sparse.csr_matrix(sample)

In [143]:
similarities = cosine_similarity(sample_sparse)

In [127]:
## 각 행마다 2번째부터 11번째로 큰 수의 위치 찾아서 그걸 데이터 프레임에 저장

In [152]:
result = pd.DataFrame(columns=["pcode","recommend_pcode"])

for i in range(0, len(Total_label)):
    row = similarities[i]
    row[i] = 0
    count = 0

    while(count<10):
        index = row.argmax()
        result = result.append({"pcode": Total_label[i],"recommend_pcode": Total_label[index]}, ignore_index=True)
        row[index] = 0
        count = count + 1
        
print("complete")

In [155]:
result.to_csv("recommend_result.csv", index=False)